In [28]:
import requests

# Set your ClickUp API token
api_token = 'pk_14892986_9S7HWBH7RNGVCC5XHYS45ZT9RLQEUKK7'

# Set the space and view details
space_name = 'Sales Engineering'
team_name = 'Arkon Data'
view_name = 'Table'

# Get the space ID
headers = {'Authorization': api_token}
response = requests.get('https://api.clickup.com/api/v2/team', headers=headers)
teams = response.json()['teams']
team_id = next((team['id'] for team in teams if team['name'] == team_name), None)

team_id

'2290945'

In [29]:
# Get the space ID
response = requests.get(f'https://api.clickup.com/api/v2/team/{team_id}/space', headers=headers)
spaces = response.json()['spaces']
space_id = next((space['id'] for space in spaces if space['name'] == space_name), None)
space_id


'48591090'

In [30]:
# Get the view ID
response = requests.get(f'https://api.clickup.com/api/v2/space/{space_id}/view', headers=headers)
views = response.json()['views']
view_id = next((view['id'] for view in views if view['name'] == view_name), None)
view_id

'25x81-21625'

In [36]:
# Get the table data
response = requests.get(f'https://api.clickup.com/api/v2/view/{view_id}/task?page=0', headers=headers)
response1 = requests.get(f'https://api.clickup.com/api/v2/view/{view_id}/task?page=1', headers=headers)

table_data = response.json()
table_data1 = response1.json()

# Process and use the table data as needed
tasks = table_data['tasks']
tasks2 = table_data1['tasks']

print(len(tasks))
print(len(tasks2))
tasks.extend(tasks2)

print(len(tasks))

30
13
43


In [37]:
import pandas as pd
import json

tasks = table_data['tasks']
df = pd.DataFrame()

import pandas as pd

data = [
    {'id': 'bb9781e6-99e1-4075-abc4-eb7a00bf383a', 'name': '1-Lead', 'color': '#02BCD4', 'orderindex': 0},
    {'id': 'a9fe9a03-5f75-4fd6-96f6-f38fbdaa66be', 'name': '2-Qualified Lead', 'color': '#04A9F4', 'orderindex': 1},
    {'id': '93a39340-6bd2-45d0-bb2e-36e8d6671207', 'name': '3-Lead Discovery', 'color': '#ff7800', 'orderindex': 2},
    {'id': '86edcceb-45b1-4754-8763-b6496d21014e', 'name': '4a-Assessment', 'color': '#f9d900', 'orderindex': 3},
    {'id': 'be6379ac-de64-4117-8e58-9a4729cf0242', 'name': '4b-PoC', 'color': '#f9d900', 'orderindex': 4},
    {'id': '96435eda-d6fd-4138-9140-63458dc6871b', 'name': '5-Proposal', 'color': '#0231E8', 'orderindex': 5},
    {'id': '208e3fb5-804c-461c-b384-356fd79400a7', 'name': '6-KT', 'color': '#1bbc9c', 'orderindex': 6},
    {'id': 'f7a30024-1861-4a3c-8966-02b950590561', 'name': '7-Followup', 'color': '#9b59b6', 'orderindex': 7},
    {'id': 'da972be7-bb47-41e9-b6dd-7ecaff386868', 'name': '8-Dev', 'color': '#667684', 'orderindex': 8},
    {'id': '103279ac-a6d4-485f-80aa-921b25465c77', 'name': '0-Onhold', 'color': '#800000', 'orderindex': 9}
]

cat = pd.DataFrame(data)
selected_columns = ['name', 'orderindex']
df_selected = cat[selected_columns]
df_selected = df_selected.rename(columns={'name': 'status_crm'})
print(df_selected)
print(df)


         status_crm  orderindex
0            1-Lead           0
1  2-Qualified Lead           1
2  3-Lead Discovery           2
3     4a-Assessment           3
4            4b-PoC           4
5        5-Proposal           5
6              6-KT           6
7        7-Followup           7
8             8-Dev           8
9          0-Onhold           9
Empty DataFrame
Columns: []
Index: []


In [38]:
import datetime

custom_field_name = 'Status CRM (test)'
task_dicts = []  # List to store task dictionaries
for task in tasks:
    task_id = task['id']
    name = task['name']
    start_date = task.get('start_date', None)
    time_tracked = task.get('time_tracked', 0)
    due_date = task.get('due_date', None)
    points = task['points']
    assignee = task['assignees'][0]['username'] if task['assignees'] else None
    list = task['list']['name'] if task['list'] else None
    time_estimate = task.get('time_estimate',0)

    if time_tracked is None:
        time_tracked = 0
    if time_estimate is None:
        time_estimate = 0
    
    if start_date is None:
        start_date = None
    else:
        start_date = datetime.datetime.fromtimestamp(int(start_date)/1000).strftime('%Y-%m-%d')

    if due_date is None:
        due_date = None
    else:
        due_date = datetime.datetime.fromtimestamp(int(due_date)/1000).strftime('%Y-%m-%d')
        

    #start_date = datetime.datetime.fromtimestamp(start_date).strftime('%Y-%m-%d')

    # Extract other fields as needed
    custom_fields = task['custom_fields']
    target_field = next((field for field in custom_fields if field['name'] == custom_field_name), None)
    if target_field is None:
        print(f"Custom field '{custom_field_name}' not found.")
        value = None
    else:
        value = target_field.get('value')

    # Create a dictionary with the extracted information
    task_dict = {
        'task_id': task_id,
        'name': name,
        'start_date': start_date,
        'value': value,
        'date': datetime.date.today().isoformat(),
        'time_tracked': int(time_tracked)/3600000,
        'due_date': due_date,
        'points': points,
        'assignee': assignee,
        'time_estimate': int(time_estimate)/3600000,
        'list': list
    }
    task_dicts.append(task_dict)

# Append the dictionary as a new row to the dataframe
df = pd.concat([df, pd.DataFrame(task_dicts)])

# Print the resulting dataframe
print(df)


      task_id                                               name  start_date  \
0   865c0ybtr                                       GKM Viaticos        None   
1   865c3udrk                                        Acapulco AI        None   
2   865cdnckn                                              SOLCA        None   
3   865cdnf39                                               APLI        None   
4   865cdnfe5                   FLEET MANAGEMENT (GRUPO  MODELO)        None   
5   865cdng99                                     GLOBAL TRADERS        None   
6   865cfh6zc  Seguimiento a propuesta/liberación de presupue...        None   
7   865cg9jv1         Contactar a Eduardo para agendar junta (I)        None   
8   865cjb5m2                                   GB - ECOSISTEMAS        None   
9   865cjb5td                            Rodolfo Cruz Echeverría        None   
10  865cjb714                             GB RIESGOS DE LIQUIDEZ        None   
11  865cjb71e                         GB

In [39]:
df_merged = df.merge(df_selected, left_on='value', right_on='orderindex', how='left')
print(df_merged.columns)

Index(['task_id', 'name', 'start_date', 'value', 'date', 'time_tracked',
       'due_date', 'points', 'assignee', 'time_estimate', 'list', 'status_crm',
       'orderindex'],
      dtype='object')


In [40]:
selected_columns = ['list', 'name', 'status_crm', 'assignee','start_date','due_date','time_estimate','time_tracked','points']
df_merged = df_merged[selected_columns]
df_merged.to_csv('output.csv', index=False)


In [41]:
pd.set_option('display.max_rows', None)  # Set option to display all rows
print(df_merged)

                                list  \
0                                GKM   
1                        ACAPULCO AI   
2                              SOLCA   
3                               APLI   
4    FLEET MANAGEMENT (GRUPO MODELO)   
5                     GLOBAL TRADERS   
6                     GLOBAL TRADERS   
7             REPORTES ACTIVOS FIJOS   
8                             CRM V2   
9                             CRM V2   
10                            CRM V2   
11                            CRM V2   
12                            JOFROY   
13                            JOFROY   
14                    GB-ECOSISTEMAS   
15               UNIDADES ECONOMICAS   
16                       GENOMMA LAB   
17                  LOGISTICA BARCEL   
18        GB-PENSIONES E INVERSIONES   
19        GB-PENSIONES E INVERSIONES   
20                    GB-ECOSISTEMAS   
21             GB-RIESGO DE LIQUIDEZ   
22                             DRACO   
23    DEBRIEF LOGISTICA CEDIS BARCEL   
